In [9]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/thesis/graph_neural_network/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/thesis/graph_neural_network


In [4]:
!pip install torchmetrics
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124

import torch
import os
print("PyTorch has version {}".format(torch.__version__))

srs_url = f"https://pytorch-geometric.com/whl/torch-{torch.__version__}.html"
!pip install torch_scatter -f $srs_url
!pip install torch_sparse -f $srs_url
!pip install torch_cluster -f $srs_url
!pip install torch_spline_conv -f $srs_url
!pip install torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 23.0 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 122.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 112.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/1

09-06 16:17:42 - DEBUG - pydot - pydot initializing
09-06 16:17:42 - DEBUG - pydot - pydot 3.0.4
09-06 16:17:42 - DEBUG - pydot.dot_parser - pydot dot_parser module initializing
09-06 16:17:42 - DEBUG - pydot.core - pydot core module initializing


PyTorch has version 2.4.0+cu124
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 63.3 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.2 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 995.7/995.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.3 MB/s eta 0:00:00


In [10]:
import logging

import sys
import itertools
import yaml
import subprocess
import os
import logging

# Path to the config file and train script
config_path = "config.yaml"
train_script = "train.py"


for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.DEBUG,
    datefmt='%m-%d %H:%M:%S',
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s"
)
logger = logging.getLogger(__name__)


In [11]:
### Change config

new_config = {
    'epochs': 50,
    "val_size": 0.2,
    "test_size": 0.2,
    "seed": 100,
    'n_layer': 2,
    'learning_rate': 0.001,
    'item_emb_dim': 128,
    'user_emb_dim': 128,
    "negative_sampling_method": "pairwise_random",
    "recon_loss": "bpr", # "binary", "bpr", "bce"
    'feature_linear_dim': 128,
    'hidden_channels': 128,
    'latent_dim': 64,
    'kl_beta': 0.5,
    "kl_warmup_epoch" : 30,
    'eval_interval': 1,
    'variational': False,
    'save_model': True
}


with open(config_path, "r") as f:
    config = yaml.safe_load(f)

config.update(new_config)

temp_config_path = f"temp_config.yaml"
with open(temp_config_path, "w") as f:
    yaml.dump(config, f)

In [12]:

process = subprocess.Popen(
    ["python", "-u", train_script, "--config", temp_config_path],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Stream output line by line
for line in process.stdout:
    print(line, end="")  # live printing in notebook

process.wait()  # wait for completion


INFO:__main__:Configuration:
INFO:__main__:{'batch_method': 'binary_link_neighbors',
 'batch_norm': False,
 'batch_size': 128,
 'book_features': [{'degree_log_transform': True,
                    'features': ['degree', 'coreness'],
                    'name': 'topo'},
                   {'name': 'textual_desc'},
                   {'aggr_fn': 'mean', 'name': 'textual_reviews'}],
 'books_filename': 'books_filtered.parquet',
 'coreness_k': 10,
 'decoder': 'inner_product_decoder',
 'descriptions_filename': 'descriptions_filtered.parquet',
 'dir': '../data_sample',
 'disjoint_train_ratio': 0.0,
 'dropout': 0.5,
 'edge_type': 'reads',
 'embeddings_descriptions_filename': 'embeddings_descriptions_sbert_pt.parquet',
 'embeddings_reviews_filename': 'embeddings_reviews_sbert_pt.parquet',
 'encoder': 'sage_encoder',
 'epochs': 50,
 'eval_interval': 1,
 'feature_aggr_method': 'concat',
 'feature_linear_dim': 128,
 'hidden_channels': 128,
 'interactions_filename': 'interactions.parquet',
 'item_e

1

In [7]:
model

NameError: name 'model' is not defined